In [25]:
import pandas as pd
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

def cached(func):
    cache = dict()
    
    def return_func(arg):
        if (arg in cache):
            return cache[arg]
        else:
            result = func(arg)
            cache[arg] = result
            return result
    
    return return_func

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

In [26]:
page_ids = query("""
    select
        page_repository.id,
        instance.subdomain,
        uuid.trashed
    from page_repository
        join uuid on uuid.id = page_repository.id
        join instance on page_repository.instance_id = instance.id;
""")
page_ids[0:10]

[(16256, 'de', 1),
 (16303, 'de', 1),
 (16306, 'de', 1),
 (16530, 'de', 1),
 (16569, 'de', 1),
 (16659, 'de', 1),
 (16816, 'de', 1),
 (18233, 'de', 1),
 (18340, 'de', 1),
 (18778, 'de', 1)]

In [27]:
import requests

def api_call(query, variables={}):
    headers = { "Content-Type": "application/json" }

    response = requests.post(
            "https://api.serlo.org/graphql",
            headers = headers,
            json = { "query": query, "variables": variables })

    if response.status_code not in [200, 400]:
        response.raise_for_status()

    try:
        result = response.json()

        if "errors" in result and result["errors"]:
            raise requests.HTTPError(result["errors"][0]["message"],
                                     response=response)

        return result["data"]
    except e:
        print(response.text)
        raise e 

api_call("""
    query($id: Int) {
        uuid(id: $id) {
            __typename
        }
    }
""", { "id": 1 })

{'uuid': {'__typename': 'User'}}

In [28]:
import json

def print_json(j):
    print(json.dumps(j, indent=2))


def get(obj, key):
    return None if obj == None else obj[key]
    
def navigation_of_page(page_id):
    uuid = api_call("""
        query {
            uuid(id: %s) {
                ... on Page {
                    navigation {
                        data
                    }
                }
            }
        }
    """ % page_id)["uuid"]
    
    return get(get(uuid, "navigation"), "data")

print_json(navigation_of_page(18922))

{
  "label": "\u00dcber Serlo",
  "id": 18922,
  "children": [
    {
      "label": "So funktioniert die Lernplattform",
      "id": 81862
    },
    {
      "label": "Wirkung",
      "id": 21406
    },
    {
      "label": "Team",
      "id": 21439
    },
    {
      "label": "Die Geschichte von Serlo",
      "id": 21413
    },
    {
      "label": "Partner und F\u00f6rderer",
      "id": 21456
    },
    {
      "label": "Transparenz",
      "id": 21468
    },
    {
      "label": "Spenden",
      "url": "/spenden"
    }
  ]
}


In [29]:
display(Markdown("# Not trashed pages"))

for (page_id, instance, trashed) in page_ids:
    if not trashed:
        display(Markdown(f"## Page {page_id} on {instance} (trashed={trashed}):"))
        print_json(navigation_of_page(page_id))

# Not trashed pages

## Page 18922 on de (trashed=0):

{
  "label": "\u00dcber Serlo",
  "id": 18922,
  "children": [
    {
      "label": "So funktioniert die Lernplattform",
      "id": 81862
    },
    {
      "label": "Wirkung",
      "id": 21406
    },
    {
      "label": "Team",
      "id": 21439
    },
    {
      "label": "Die Geschichte von Serlo",
      "id": 21413
    },
    {
      "label": "Partner und F\u00f6rderer",
      "id": 21456
    },
    {
      "label": "Transparenz",
      "id": 21468
    },
    {
      "label": "Spenden",
      "url": "/spenden"
    }
  ]
}


## Page 19767 on de (trashed=0):

{
  "label": "Mathematik",
  "id": 19767,
  "children": [
    {
      "label": "Alle Themen",
      "id": 5
    },
    {
      "label": "Gymnasium",
      "id": 16042
    },
    {
      "label": "Realschule",
      "id": 16157
    },
    {
      "label": "Mittelschule (Hauptschule)",
      "id": 16259
    },
    {
      "label": "FOS & BOS",
      "id": 201593
    },
    {
      "label": "Hochschule",
      "id": 44323
    },
    {
      "label": "Pr\u00fcfungen",
      "id": 83249
    },
    {
      "label": "Inhalte bearbeiten und neue Inhalte hinzuf\u00fcgen",
      "id": 19880
    },
    {
      "label": "Chat f\u00fcr Mathe-AutorInnen",
      "url": "https://community.serlo.org/channel/mathe"
    }
  ]
}


...

null


## Page 143390 on fr (trashed=0):

null


## Page 148617 on fr (trashed=0):

{
  "label": "Conseils pour apprendre",
  "id": 148617,
  "children": [
    {
      "label": "Tous les th\u00e8mes",
      "id": 204968
    }
  ]
}


In [30]:
display(Markdown("# Trashed pages"))

for (page_id, instance, trashed) in page_ids:
    if trashed:
        display(Markdown(f"## Page {page_id} on {instance} (trashed={trashed}):"))
        print_json(navigation_of_page(page_id))

# Trashed pages

## Page 16256 on de (trashed=1):

null


## Page 16303 on de (trashed=1):

null


...

null


## Page 149156 on ta (trashed=1):

null


## Page 149154 on fr (trashed=1):

null


In [31]:
taxonomy_terms = query("""
    select
        subject.id, term.name, instance.subdomain
    from term_taxonomy as subject
        join term_taxonomy as root on root.id = subject.parent_id
        join term on term.id = subject.term_id
        join instance on term.instance_id = instance.id
    where root.parent_id is null;
""")
taxonomy_terms[0:5]

[(5, 'Mathe', 'de'),
 (8, 'Blog', 'de'),
 (15465, 'Mathematik Forum', 'de'),
 (17744, 'Nachhaltigkeit', 'de'),
 (17746, 'Permakultur Forum', 'de')]

In [32]:
def taxonomy_nav(taxonomy_term_id):
    uuid = api_call("""
        query($id: Int) {
            uuid(id: $id) {
                ... on TaxonomyTerm {
                    navigation {
                        data
                    }
                }
            }
        }
    """, { "id": taxonomy_term_id})["uuid"]
    
    return get(get(uuid, "navigation"), "data")

print_json(taxonomy_nav(5))

{
  "label": "Mathematik",
  "id": 19767,
  "children": [
    {
      "label": "Alle Themen",
      "id": 5
    },
    {
      "label": "Gymnasium",
      "id": 16042
    },
    {
      "label": "Realschule",
      "id": 16157
    },
    {
      "label": "Mittelschule (Hauptschule)",
      "id": 16259
    },
    {
      "label": "FOS & BOS",
      "id": 201593
    },
    {
      "label": "Hochschule",
      "id": 44323
    },
    {
      "label": "Pr\u00fcfungen",
      "id": 83249
    },
    {
      "label": "Inhalte bearbeiten und neue Inhalte hinzuf\u00fcgen",
      "id": 19880
    },
    {
      "label": "Chat f\u00fcr Mathe-AutorInnen",
      "url": "https://community.serlo.org/channel/mathe"
    }
  ]
}


# Navigation of TaxonomyTerms

In [33]:
for (taxonomy_term_id, name, subdomain) in taxonomy_terms:
    display(Markdown(f"## Taxonomy '{subdomain}:{name}' with id {taxonomy_term_id}"))
    print_json(taxonomy_nav(taxonomy_term_id))

## Taxonomy 'de:Mathe' with id 5

{
  "label": "Mathematik",
  "id": 19767,
  "children": [
    {
      "label": "Alle Themen",
      "id": 5
    },
    {
      "label": "Gymnasium",
      "id": 16042
    },
    {
      "label": "Realschule",
      "id": 16157
    },
    {
      "label": "Mittelschule (Hauptschule)",
      "id": 16259
    },
    {
      "label": "FOS & BOS",
      "id": 201593
    },
    {
      "label": "Hochschule",
      "id": 44323
    },
    {
      "label": "Pr\u00fcfungen",
      "id": 83249
    },
    {
      "label": "Inhalte bearbeiten und neue Inhalte hinzuf\u00fcgen",
      "id": 19880
    },
    {
      "label": "Chat f\u00fcr Mathe-AutorInnen",
      "url": "https://community.serlo.org/channel/mathe"
    }
  ]
}


## Taxonomy 'de:Blog' with id 8

null


## Taxonomy 'de:Mathematik Forum' with id 15465

...

null


## Taxonomy 'fr:nouvelles matiÃ¨res' with id 141607

null


## Taxonomy 'fr:Conseils pour apprendre' with id 148619

null
